In [ ]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image

# -------------------
# Config
# -------------------
class CFG:
    img_size = 128   # smaller images for speed
    batch_size = 32
    lr = 1e-3
    epochs = 5       # fewer epochs for faster training
    device = "cuda" if torch.cuda.is_available() else "cpu"
    img_root = "./images"   # folder with images (0.jpg, 1.jpg, ...)
    csv_path = "image.house.csv"

# -------------------
# Dataset
# -------------------
class HouseDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(CFG.img_root, f"{row['image_id']}.jpg")
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.new("RGB", (CFG.img_size, CFG.img_size), (128,128,128))

        if self.transform:
            img = self.transform(img)

        numeric = row[["bed","bath","sqft","n_citi"]].values.astype(np.float32)
        y = row["price"]

        return {
            "image": img,
            "numeric": torch.tensor(numeric, dtype=torch.float),
            "target": torch.tensor(y, dtype=torch.float),
        }

# -------------------
# Model
# -------------------
class FastMultimodal(nn.Module):
    def __init__(self, num_numeric):
        super().__init__()
        # lightweight CNN
        self.cnn = models.mobilenet_v2(pretrained=True)
        self.cnn.classifier = nn.Identity()
        img_out = 1280  # mobilenet v2 output

        # small tabular network
        self.mlp = nn.Sequential(
            nn.Linear(num_numeric, 32),
            nn.ReLU()
        )

        # fusion
        self.regressor = nn.Sequential(
            nn.Linear(img_out+32, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, img, num):
        img_feat = self.cnn(img)
        num_feat = self.mlp(num)
        fused = torch.cat([img_feat, num_feat], dim=1)
        return self.regressor(fused).squeeze(1)

# -------------------
# Training
# -------------------
def train_fast():
    df = pd.read_csv(CFG.csv_path)

    # scale numeric
    scaler = StandardScaler()
    df[["bed","bath","sqft","n_citi"]] = scaler.fit_transform(df[["bed","bath","sqft","n_citi"]])

    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    transform = T.Compose([
        T.Resize((CFG.img_size, CFG.img_size)),
        T.ToTensor(),
    ])

    train_ds = HouseDataset(train_df, transform)
    val_ds = HouseDataset(val_df, transform)

    train_loader = DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=CFG.batch_size)

    model = FastMultimodal(num_numeric=4).to(CFG.device)
    opt = torch.optim.Adam(model.parameters(), lr=CFG.lr)
    criterion = nn.MSELoss()

    for epoch in range(CFG.epochs):
        model.train()
        for batch in train_loader:
            imgs = batch["image"].to(CFG.device)
            nums = batch["numeric"].to(CFG.device)
            y = batch["target"].to(CFG.device)

            opt.zero_grad()
            preds = model(imgs, nums)
            loss = criterion(preds, y)
            loss.backward()
            opt.step()

        # validation
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for batch in val_loader:
                imgs = batch["image"].to(CFG.device)
                nums = batch["numeric"].to(CFG.device)
                y = batch["target"].to(CFG.device)
                preds = model(imgs, nums)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(y.cpu().numpy())

        mae = mean_absolute_error(val_true, val_preds)
        rmse = math.sqrt(mean_squared_error(val_true, val_preds))
        print(f"Epoch {epoch+1}: MAE={mae:.2f}, RMSE={rmse:.2f}")

    return model

if __name__ == "__main__":
    train_fast()


C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100%|█████████████████████████████████████████████████████████████████████████████| 13.6M/13.6M [00:06<00:00, 2.18MB/s]


Epoch 1: MAE=689997.42, RMSE=789532.96
Epoch 2: MAE=544368.57, RMSE=665289.09
Epoch 3: MAE=352599.02, RMSE=496348.29
Epoch 4: MAE=276128.78, RMSE=401491.11
